# Quickmodel
A simple command project I'm putting together. Goal: User runs the file with a specific search term, and the program returns a pickled ML model that is trained to recognize images of the search term. 

In [7]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
import matplotlib.pyplot as plt
import os
img_dir = os.listdir('../data/')
img_count = len(img_dir)
type(img_dir)

list

First, let's generate our dataset from our existing files. Luckily, ``keras`` has a nice way of implementing this from their Image library.

In [10]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.3)
IMG_HEIGHT = 256
IMG_WIDTH = 256
STEPS_PER_EPOCH = np.ceil(img_count/BATCH_SIZE)

train_generator = image_generator.flow_from_directory(directory='../data',
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = ['Dogs'], #should come from script parameter
                                                     subset='training') 
validation_generator = image_generator.flow_from_directory(directory='../data',
                                                          target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                          classes= ['Dogs'],
                                                          subset='validation')

Found 11 images belonging to 1 classes.
Found 4 images belonging to 1 classes.


Now that we have our dataset, we need to construct a model and train it.

In [33]:
model = keras.models.Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
